In [30]:
# Import all required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import io
import base64
import json
import sys
import warnings
from datetime import datetime
import asyncio
import nest_asyncio
from typing import Dict, List, Any, Optional, Tuple

# ML imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

# UI imports - ONLY ipywidgets
import ipywidgets as widgets
from ipywidgets import (
    Layout, VBox, HBox, Tab, Output, HTML, Button, Label, 
    Text, Textarea, Checkbox, Dropdown, FloatSlider, IntSlider,
    Select, SelectMultiple, FileUpload, RadioButtons, Accordion,
    IntRangeSlider, FloatRangeSlider, GridBox, Box, AppLayout
)

# Display utilities
from IPython.display import display, clear_output, HTML, Javascript, Markdown
import IPython

# Apply nest_asyncio for async operations in Jupyter
nest_asyncio.apply()
warnings.filterwarnings('ignore')

# Global state
state = {
    'df': None,
    'filtered_df': None,
    'history': [],
    'history_index': -1,
    'theme': 'dark',
    'current_filters': {},
    'uploaded_file': None,
    'visualizations': {},
    'ml_model': None,
    'ml_results': None
}

# Initialize session history
state['history'].append({"action": "Initial State", "timestamp": datetime.now().isoformat()})
state['history_index'] = 0



In [31]:
# Inject futuristic neon CSS
display(HTML("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700&family=Exo+2:wght@300;400;600&display=swap');
    
    :root {
        --neon-cyan: #00f3ff;
        --neon-magenta: #ff00ff;
        --neon-violet: #9d00ff;
        --dark-bg: #0a0a1a;
        --darker-bg: #050510;
        --card-bg: rgba(16, 18, 40, 0.7);
        --glass-bg: rgba(255, 255, 255, 0.05);
        --text-primary: #ffffff;
        --text-secondary: #a0a0c0;
    }
    
    body {
        font-family: 'Exo 2', sans-serif;
        background-color: var(--dark-bg);
        color: var(--text-primary);
        margin: 0;
        padding: 10px;
    }
    
    .jupyter-widgets {
        font-family: 'Exo 2', sans-serif !important;
    }
    
    .neon-text {
        font-family: 'Orbitron', sans-serif;
        text-shadow: 0 0 10px var(--neon-cyan),
                     0 0 20px var(--neon-cyan),
                     0 0 30px var(--neon-cyan);
        color: var(--neon-cyan) !important;
    }
    
    .magenta-text {
        color: var(--neon-magenta) !important;
        text-shadow: 0 0 5px rgba(255, 0, 255, 0.5);
    }
    
    .glass-card {
        background: var(--card-bg) !important;
        backdrop-filter: blur(10px);
        border: 1px solid rgba(0, 243, 255, 0.2) !important;
        border-radius: 12px !important;
        box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3) !important;
        padding: 20px;
        margin: 10px 0;
    }
    
    .neon-button {
        background: linear-gradient(45deg, var(--neon-violet), var(--neon-magenta)) !important;
        border: none !important;
        color: white !important;
        font-weight: 600 !important;
        text-transform: uppercase !important;
        letter-spacing: 1px !important;
        transition: all 0.3s ease !important;
        margin: 5px !important;
    }
    
    .neon-button:hover {
        box-shadow: 0 0 15px var(--neon-magenta),
                    0 0 25px var(--neon-magenta) !important;
        transform: scale(1.05) !important;
    }
    
    .secondary-button {
        background: rgba(0, 243, 255, 0.1) !important;
        border: 1px solid var(--neon-cyan) !important;
        color: var(--neon-cyan) !important;
    }
    
    .shimmer {
        background: linear-gradient(90deg, 
            rgba(0, 243, 255, 0.1) 25%, 
            rgba(157, 0, 255, 0.2) 50%, 
            rgba(0, 243, 255, 0.1) 75%);
        background-size: 200% 100%;
        animation: shimmer 1.5s infinite;
    }
    
    @keyframes shimmer {
        0% { background-position: -200% 0; }
        100% { background-position: 200% 0; }
    }
    
    .pulse {
        animation: pulse 2s infinite;
    }
    
    @keyframes pulse {
        0%, 100% { opacity: 1; }
        50% { opacity: 0.7; }
    }
    
    .fade-in {
        animation: fadeIn 0.5s ease-in;
    }
    
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(10px); }
        to { opacity: 1; transform: translateY(0); }
    }
    
    .data-table {
        background: rgba(10, 10, 26, 0.8) !important;
        border: 1px solid rgba(0, 243, 255, 0.2) !important;
        color: white !important;
    }
    
    .data-table th {
        background: rgba(0, 243, 255, 0.1) !important;
        color: var(--neon-cyan) !important;
        font-weight: 600 !important;
        padding: 10px !important;
    }
    
    .data-table td {
        border-bottom: 1px solid rgba(0, 243, 255, 0.1) !important;
        padding: 8px !important;
    }
    
    .slider-track {
        background: linear-gradient(to right, var(--neon-violet), var(--neon-magenta)) !important;
    }
    
    .tab-active {
        border-bottom: 2px solid var(--neon-cyan) !important;
        color: var(--neon-cyan) !important;
    }
    
    .stat-card {
        background: linear-gradient(135deg, 
            rgba(0, 243, 255, 0.1), 
            rgba(157, 0, 255, 0.1)) !important;
        border-left: 4px solid var(--neon-cyan) !important;
        padding: 15px;
        margin: 10px;
        border-radius: 8px;
    }
    
    .loading-placeholder {
        height: 200px;
        display: flex;
        align-items: center;
        justify-content: center;
        background: rgba(16, 18, 40, 0.5);
        border-radius: 8px;
        border: 2px dashed rgba(0, 243, 255, 0.3);
        color: var(--neon-cyan);
        font-weight: bold;
    }
    
    .insight-card {
        border-left: 4px solid var(--neon-magenta) !important;
        padding: 15px;
        margin: 10px;
        background: rgba(255, 0, 255, 0.05);
        border-radius: 8px;
    }
    
    .warning-card {
        border-left: 4px solid #ff9900 !important;
        padding: 15px;
        margin: 10px;
        background: rgba(255, 153, 0, 0.05);
        border-radius: 8px;
    }
    
    /* Scrollbar styling */
    ::-webkit-scrollbar {
        width: 8px;
        height: 8px;
    }
    
    ::-webkit-scrollbar-track {
        background: rgba(0, 0, 0, 0.2);
        border-radius: 4px;
    }
    
    ::-webkit-scrollbar-thumb {
        background: linear-gradient(45deg, var(--neon-violet), var(--neon-magenta));
        border-radius: 4px;
    }
    
    /* Widget styling */
    .widget-slider .ui-slider-track {
        background: linear-gradient(to right, var(--neon-violet), var(--neon-magenta)) !important;
    }
    
    .widget-dropdown {
        background: rgba(16, 18, 40, 0.8) !important;
        color: white !important;
        border: 1px solid rgba(0, 243, 255, 0.3) !important;
    }
    
    .widget-text {
        background: rgba(16, 18, 40, 0.8) !important;
        color: white !important;
        border: 1px solid rgba(0, 243, 255, 0.3) !important;
    }
    
    .widget-checkbox {
        color: white !important;
    }
    
    .widget-label {
        color: var(--neon-cyan) !important;
        font-weight: 600 !important;
    }
    
    /* Tab styling */
    .p-TabBar-tab {
        color: #a0a0c0 !important;
        background: rgba(10, 10, 26, 0.7) !important;
        border: 1px solid rgba(0, 243, 255, 0.2) !important;
        border-bottom: none !important;
        margin-right: 5px !important;
        padding: 8px 15px !important;
        font-family: 'Exo 2', sans-serif !important;
        font-weight: 600 !important;
    }
    
    .p-TabBar-tab.p-mod-current {
        color: #00f3ff !important;
        background: rgba(0, 243, 255, 0.1) !important;
        border-color: #00f3ff !important;
        box-shadow: 0 0 10px rgba(0, 243, 255, 0.3) !important;
    }
    
    .p-TabBar-tab:hover:not(.p-mod-current) {
        color: #ff00ff !important;
        background: rgba(255, 0, 255, 0.1) !important;
        border-color: rgba(255, 0, 255, 0.3) !important;
    }
    
    .p-TabPanel {
        background: rgba(16, 18, 40, 0.7) !important;
        border: 1px solid rgba(0, 243, 255, 0.2) !important;
        border-top: none !important;
        padding: 20px !important;
    }
    
    /* Header styling */
    .dashboard-header {
        background: linear-gradient(135deg, rgba(10, 10, 26, 0.9), rgba(16, 18, 40, 0.9));
        border-radius: 12px;
        padding: 20px;
        margin-bottom: 20px;
        border: 1px solid rgba(0, 243, 255, 0.3);
        box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
        text-align: center;
    }
</style>
"""))

# Helper functions
def add_to_history(action, data=None):
    """Add action to history stack"""
    state['history'].append({
        "action": action,
        "timestamp": datetime.now().isoformat(),
        "data": data if data else {},
        "index": len(state['history'])
    })
    state['history_index'] = len(state['history']) - 1

def undo_action():
    """Undo last action"""
    if state['history_index'] > 0:
        state['history_index'] -= 1
        return True
    return False

def redo_action():
    """Redo action"""
    if state['history_index'] < len(state['history']) - 1:
        state['history_index'] += 1
        return True
    return False

def detect_delimiter(file_content):
    """Detect CSV delimiter from file content"""
    sample = file_content[:1000]
    delimiters = [',', ';', '\t', '|']
    counts = {delim: sample.count(delim) for delim in delimiters}
    return max(counts, key=counts.get)

def create_stat_card(title, value, color="cyan"):
    """Create a statistic card"""
    color_class = "neon-text" if color == "cyan" else "magenta-text"
    return VBox([
        HTML(f"<div style='font-size: 2rem;' class='{color_class}'>{value}</div>"),
        HTML(f"<div style='font-size: 0.9rem; color: var(--text-secondary);'>{title}</div>")
    ], layout=Layout(
        padding='15px',
        margin='5px',
        border='1px solid rgba(0, 243, 255, 0.2)',
        border_radius='8px',
        background='rgba(16, 18, 40, 0.5)'
    ))

def create_loading_placeholder(height=100):
    """Create a loading placeholder"""
    return HTML(f"""
    <div class="loading-placeholder" style="height: {height}px;">
        <div class="shimmer" style="width: 100%; height: 100%; border-radius: 8px;"></div>
    </div>
    """)

def create_alert(message, alert_type="info"):
    """Create an alert message"""
    colors = {
        "info": "#00f3ff",
        "success": "#00ff00",
        "warning": "#ff9900",
        "error": "#ff0000"
    }
    color = colors.get(alert_type, "#00f3ff")
    
    return HTML(f"""
    <div style="
        background: rgba({int(color[1:3], 16)}, {int(color[3:5], 16)}, {int(color[5:7], 16)}, 0.1);
        border-left: 4px solid {color};
        padding: 15px;
        margin: 10px 0;
        border-radius: 8px;
        color: white;
    ">
        <strong>{alert_type.upper()}:</strong> {message}
    </div>
    """)

# Create output widgets for different sections
upload_output = Output()
profile_output = Output()
filter_output = Output()
viz_output = Output()
insights_output = Output()
ml_output = Output()
history_output = Output()
export_output = Output()
api_output = Output()



In [32]:
# Create dashboard header
header = HTML("""
<div class="dashboard-header fade-in">
    <h1 class="neon-text" style="font-size: 2.5rem; margin: 0;">
        ⚡ NeonFlux Dashboard
    </h1>
    <p style="color: var(--text-secondary); font-size: 1.2rem; margin: 10px 0 20px 0;">
        Futuristic Data Insights Platform
    </p>
    
    <div style="display: flex; justify-content: center; gap: 15px; flex-wrap: wrap;">
        <div style="display: flex; align-items: center;">
            <span style="color: var(--neon-cyan); margin-right: 10px;">⚡</span>
            <span style="color: white;">Real-time Analytics</span>
        </div>
        <div style="display: flex; align-items: center;">
            <span style="color: var(--neon-magenta); margin-right: 10px;">🤖</span>
            <span style="color: white;">AI-Powered Insights</span>
        </div>
        <div style="display: flex; align-items: center;">
            <span style="color: var(--neon-violet); margin-right: 10px;">📊</span>
            <span style="color: white;">Interactive Visualizations</span>
        </div>
    </div>
</div>
""")

# Create status bar
def create_status_bar():
    """Create a status bar showing current state"""
    if state['df'] is not None:
        df = state['df']
        status_html = f"""
        <div style="
            background: rgba(0, 243, 255, 0.1);
            padding: 10px 20px;
            border-radius: 8px;
            border-left: 4px solid #00f3ff;
            margin: 10px 0;
            display: flex;
            justify-content: space-between;
            align-items: center;
            flex-wrap: wrap;
        ">
            <div style="display: flex; align-items: center; gap: 20px;">
                <div>
                    <span style="color: #00f3ff;">📊 Data:</span>
                    <span style="color: white; margin-left: 5px;">
                        {len(df):,} rows × {len(df.columns)} columns
                    </span>
                </div>
                <div>
                    <span style="color: #ff00ff;">📁 File:</span>
                    <span style="color: white; margin-left: 5px;">
                        {state.get('uploaded_file', 'Unknown')}
                    </span>
                </div>
                <div>
                    <span style="color: #9d00ff;">🔄 Actions:</span>
                    <span style="color: white; margin-left: 5px;">
                        {len(state['history'])} recorded
                    </span>
                </div>
            </div>
            <div>
                <span style="color: #00ff00;">✅ Ready for analysis</span>
            </div>
        </div>
        """
    else:
        status_html = """
        <div style="
            background: rgba(255, 153, 0, 0.1);
            padding: 10px 20px;
            border-radius: 8px;
            border-left: 4px solid #ff9900;
            margin: 10px 0;
            display: flex;
            justify-content: space-between;
            align-items: center;
        ">
            <div>
                <span style="color: #ff9900;">⚠ No data loaded</span>
                <span style="color: var(--text-secondary); margin-left: 10px;">
                    Upload a file or load sample data to begin
                </span>
            </div>
            <div>
                <span style="color: #00f3ff;">📤 Waiting for data</span>
            </div>
        </div>
        """
    return HTML(status_html)

# Create quick action buttons
quick_actions = HBox([
    Button(
        description='📊 Load Sample Data',
        button_style='success',
        layout=Layout(width='200px', height='40px'),
        style={'font_weight': 'bold'}
    ),
    Button(
        description='🚀 Test All Features',
        button_style='info',
        layout=Layout(width='200px', height='40px'),
        style={'font_weight': 'bold'}
    ),
    Button(
        description='🔄 Reset Dashboard',
        button_style='warning',
        layout=Layout(width='200px', height='40px'),
        style={'font_weight': 'bold'}
    )
])

# Add custom styling to buttons
display(HTML("""
<style>
    .widget-button {
        background: linear-gradient(45deg, #9d00ff, #ff00ff) !important;
        border: none !important;
        color: white !important;
        font-weight: bold !important;
        margin: 5px !important;
        transition: all 0.3s !important;
    }
    
    .widget-button:hover {
        box-shadow: 0 0 15px #ff00ff, 0 0 25px #ff00ff !important;
        transform: scale(1.05) !important;
    }
    
    button[description='📊 Load Sample Data'] {
        background: linear-gradient(45deg, #9d00ff, #6a00ff) !important;
    }
    
    button[description='🚀 Test All Features'] {
        background: linear-gradient(45deg, #00f3ff, #0099ff) !important;
    }
    
    button[description='🔄 Reset Dashboard'] {
        background: linear-gradient(45deg, #ff00ff, #ff0099) !important;
    }
</style>
"""))

# Display header components
#display(header)
#status_bar = create_status_bar()
#display(status_bar)
#display(quick_actions)

#print("✅ Dashboard header and navigation created!")

In [33]:
# Fixed upload cell — copy/paste this cell and run
import io
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import display, clear_output, HTML as IPyHTML
from ipywidgets import FileUpload, Button, HBox, VBox, Layout, HTML, Output

# ---------- Defensive: reuse existing globals if they exist ----------
# If your notebook already has these defined earlier, the following will keep them.
globals().setdefault('state', {'df': None, 'filtered_df': None, 'uploaded_file': None, 'history': [], 'history_index': 0})
globals().setdefault('upload_output', Output())

# ---------- Simple helper implementations (safe defaults) ----------
def create_alert(message, kind="info"):
    """Return a simple HTML alert widget. kind: 'info'|'success'|'warning'|'error'."""
    color = {"info":"#2196f3","success":"#00c853","warning":"#ffab00","error":"#ff1744"}.get(kind, "#2196f3")
    html = f"""
    <div style="border-left:4px solid {color}; background: rgba(0,0,0,0.35); padding:10px; border-radius:6px;">
        <div style="color:{color}; font-weight:600; margin-bottom:6px;">{message}</div>
    </div>
    """
    return HTML(html)

def create_loading_placeholder(height=80):
    """Small placeholder while processing."""
    return HTML(f"<div style='height:{height}px; display:flex; align-items:center;'><em>Processing...</em></div>")

def add_to_history(action, meta=None):
    h = state.setdefault('history', [])
    h.append({'timestamp': datetime.now().isoformat(), 'action': action, 'meta': meta or {}})
    # cap history
    state['history'] = h[-500:]

def create_status_bar():
    """Return a small HTML status bar widget (string inside HTML.value)."""
    rows = len(state['df']) if state.get('df') is not None else 0
    cols = len(state['df'].columns) if state.get('df') is not None else 0
    html = f"<div style='padding:6px; font-size:0.9rem; color:var(--text-secondary)'>Status: rows={rows:,} cols={cols}</div>"
    return HTML(html)

# create a status_bar widget (used/updated in code)
status_bar = HTML(create_status_bar().value)

# ---------- Outputs and UI elements ----------
upload_output = globals().get('upload_output', Output())

#print("📁 Creating file upload system...")

# FileUpload widget
file_upload = FileUpload(
    accept='.csv,.xlsx,.json,.txt,.gz,.zip',
    multiple=False,
    description='Upload File',
    layout=Layout(width='300px')
)

# Upload button
upload_button = Button(
    description='📤 Process Upload',
    button_style='primary',
    layout=Layout(width='150px', height='40px'),
    style={'font_weight': 'bold'}
)

# File info display
file_info = HTML("<div style='color: var(--text-secondary);'>No file selected</div>")

# Quick action buttons container (quick_actions)
load_btn = Button(description="Load Sample Data", button_style="")
test_btn = Button(description="Test All Features")
reset_btn = Button(description="Reset Dashboard", button_style="warning")
quick_actions = HBox([load_btn, test_btn, reset_btn], layout=Layout(margin='12px 0'))

# Upload section layout
upload_section = VBox([
    HTML("<h3 class='neon-text'>📁 Data Upload Zone</h3>"),
    HTML("<p style='color: var(--text-secondary);'>Supported formats: CSV, Excel, JSON, TXT (compressed: .gz, .zip)</p>"),
    HBox([file_upload, upload_button], layout=Layout(margin='20px 0')),
    file_info,
    status_bar,
    upload_output,
    quick_actions
], layout=Layout(
    padding='20px',
    border='1px solid rgba(0, 243, 255, 0.2)',
    border_radius='12px',
    background='rgba(16, 18, 40, 0.3)'
))

# ---------- Processing logic ----------
def _bytes_from_upload_value(uploaded_entry):
    """Normalize uploaded entry to bytes and filename."""
    # uploaded_entry likely is a dict from FileUpload.value like {'name':..., 'type':..., 'size':..., 'content': b'...'}
    if isinstance(uploaded_entry, dict):
        # some frontends encode content as memoryview
        content = uploaded_entry.get('content')
        if isinstance(content, memoryview):
            content = content.tobytes()
        filename = uploaded_entry.get('name') or uploaded_entry.get('filename') or ""
        return content, filename
    # fallback: if the upload widget gave a mapping keyed by filename
    return None, None

def process_upload(button=None):
    """Process uploaded file"""
    with upload_output:
        clear_output()
        try:
            if not file_upload.value:
                display(create_alert("Please select a file first", "warning"))
                return

            # FileUpload.value is a dict keyed by filename in ipywidgets, or list-like in some versions
            # Normalize to a single uploaded file dict
            entry = None
            if isinstance(file_upload.value, dict):
                # take first value
                entry = list(file_upload.value.values())[0]
            elif isinstance(file_upload.value, (list, tuple)) and file_upload.value:
                entry = file_upload.value[0]
            else:
                entry = file_upload.value

            content, filename = _bytes_from_upload_value(entry)
            if content is None:
                display(create_alert("Unable to read uploaded file content.", "error"))
                return

            # Show loading placeholder
            display(create_loading_placeholder(100))

            # Ensure content is bytes
            if isinstance(content, str):
                # convert string to bytes
                content = content.encode('utf-8')

            # Process based on extension (lower-cased)
            fname = filename.lower()
            df = None

            if fname.endswith('.csv') or fname.endswith('.txt'):
                # Try different delimiters; BytesIO works for pandas
                tried = False
                for delimiter in [',', ';', '\t', '|']:
                    try:
                        df = pd.read_csv(io.BytesIO(content), delimiter=delimiter)
                        tried = True
                        break
                    except Exception:
                        df = None
                if not tried:
                    # final fallback attempt
                    try:
                        df = pd.read_csv(io.BytesIO(content))
                    except Exception as e:
                        raise e

            elif fname.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(io.BytesIO(content))

            elif fname.endswith('.json'):
                # try bytes -> text decode, then read_json
                try:
                    txt = content.decode('utf-8')
                    df = pd.read_json(io.StringIO(txt))
                except Exception:
                    # try reading directly from BytesIO
                    df = pd.read_json(io.BytesIO(content), orient='records')

            else:
                # unknown extension: try CSV first then Excel
                try:
                    df = pd.read_csv(io.BytesIO(content))
                except Exception:
                    df = pd.read_excel(io.BytesIO(content))

            # If df still None, raise
            if df is None:
                raise ValueError("Could not parse file into a DataFrame. Try a different format or encoding.")

            # Store in state
            state['df'] = df
            state['filtered_df'] = df.copy()
            state['uploaded_file'] = filename

            # Add to history
            add_to_history(f"Uploaded file: {filename}", {
                "rows": len(df),
                "columns": len(df.columns),
                "file_size": len(content)
            })

            # Update file info
            file_info.value = f"""
            <div style="
                background: rgba(0, 255, 0, 0.06);
                padding: 12px;
                border-radius: 8px;
                border-left: 4px solid #00c853;
                margin: 10px 0;
            ">
                <h4 style="color: #00c853; margin: 0 0 8px 0;">✅ File Uploaded Successfully!</h4>
                <p style="margin: 4px 0;"><strong>Filename:</strong> {filename}</p>
                <p style="margin: 4px 0;"><strong>Rows:</strong> {len(df):,}</p>
                <p style="margin: 4px 0;"><strong>Columns:</strong> {len(df.columns)}</p>
                <p style="margin: 4px 0;"><strong>File size:</strong> {len(content)/1024:.1f} KB</p>
            </div>
            """

            # Show data preview and success alert
            clear_output()
            display(create_alert(f"File '{filename}' loaded successfully! ({len(df):,} rows × {len(df.columns)} columns)", "success"))

            # Show preview table (first 10 rows)
            preview_html = df.head(10).to_html(classes='data-table', index=False)
            display(IPyHTML(f"""
            <div style="margin: 20px 0;">
                <h4 style="color: var(--neon-cyan);">Data Preview (First 10 rows):</h4>
                <div style="overflow-x: auto; max-height: 400px;">
                    {preview_html}
                </div>
            </div>
            """))

            # Update status bar
            status_bar.value = create_status_bar().value

        except Exception as e:
            clear_output()
            display(create_alert(f"Error processing file: {str(e)}", "error"))
            file_info.value = f"""
            <div style="
                background: rgba(255, 0, 0, 0.06);
                padding: 12px;
                border-radius: 8px;
                border-left: 4px solid #ff3d00;
            ">
                <h4 style="color: #ff3d00; margin: 0;">❌ Upload Failed</h4>
                <p style="margin: 5px 0;">Error: {str(e)}</p>
                <p style="margin: 5px 0;">Try a different file format or check file encoding.</p>
            </div>
            """

# Connect button to handler
upload_button.on_click(process_upload)

# ---------- Sample data loader (connected to quick_actions Load Sample Data button) ----------
def load_sample_data(button=None):
    """Load sample data for demonstration"""
    with upload_output:
        clear_output()
        display(create_loading_placeholder(100))
        try:
            np.random.seed(42)
            n_samples = 1000
            sample_df = pd.DataFrame({
                'Date': pd.date_range('2024-01-01', periods=n_samples, freq='D'),
                'Sales': np.random.normal(1000, 200, n_samples).cumsum() + np.random.randn(n_samples) * 50,
                'Temperature': np.random.normal(22, 5, n_samples),
                'Humidity': np.random.uniform(40, 85, n_samples),
                'Region': np.random.choice(['North', 'South', 'East', 'West'], n_samples),
                'Product': np.random.choice(['Widget A', 'Widget B', 'Widget C', 'Widget D'], n_samples),
                'Revenue': np.random.exponential(500, n_samples),
                'Customers': np.random.poisson(45, n_samples),
                'Rating': np.random.choice([1, 2, 3, 4, 5], n_samples, p=[0.05, 0.1, 0.15, 0.4, 0.3]),
                'Promotion': np.random.choice([0, 1], n_samples, p=[0.6, 0.4]),
                'Cost': np.random.uniform(10, 100, n_samples),
                'Profit': np.random.normal(200, 50, n_samples)
            })

            # Add missing values
            for col in ['Temperature', 'Humidity']:
                idx = np.random.choice(n_samples, size=int(n_samples * 0.05), replace=False)
                sample_df.loc[idx, col] = np.nan

            # Add outliers
            outlier_idx = np.random.choice(n_samples, size=20, replace=False)
            sample_df.loc[outlier_idx, 'Sales'] *= 3

            # Update state
            state['df'] = sample_df
            state['filtered_df'] = sample_df.copy()
            state['uploaded_file'] = "sample_dataset.csv"

            # Add to history
            add_to_history("Loaded sample dataset", {
                "rows": len(sample_df),
                "columns": len(sample_df.columns),
                "description": "Sample sales data with multiple features"
            })

            # Update file info
            file_info.value = f"""
            <div style="
                background: rgba(0, 255, 0, 0.06);
                padding: 12px;
                border-radius: 8px;
                border-left: 4px solid #00c853;
                margin: 10px 0;
            ">
                <h4 style="color: #00c853; margin: 0 0 8px 0;">✅ Sample Data Loaded Successfully!</h4>
                <p style="margin: 4px 0;"><strong>Dataset:</strong> Sample Sales Data</p>
                <p style="margin: 4px 0;"><strong>Rows:</strong> {len(sample_df):,}</p>
                <p style="margin: 4px 0;"><strong>Columns:</strong> {len(sample_df.columns)}</p>
            </div>
            """

            # Show success message and preview
            clear_output()
            display(create_alert(f"Sample data loaded successfully! ({len(sample_df):,} rows × {len(sample_df.columns)} columns)", "success"))

            # Show preview table
            preview_html = sample_df.head(10).to_html(classes='data-table', index=False)
            display(IPyHTML(f"""
            <div style="margin: 20px 0;">
                <h4 style="color: var(--neon-cyan);">Sample Data Preview:</h4>
                <div style="overflow-x: auto; max-height: 400px;">
                    {preview_html}
                </div>
            </div>
            """))

            # Update status bar
            status_bar.value = create_status_bar().value

        except Exception as e:
            clear_output()
            display(create_alert(f"Error loading sample data: {str(e)}", "error"))

# Connect quick action buttons
load_btn.on_click(load_sample_data)

def test_all_features(button=None):
    """Test all dashboard features"""
    if state.get('df') is None:
        load_sample_data(button)
    with upload_output:
        display(create_alert("🚀 Testing all features! Switch to other tabs to see the data in action.", "info"))

test_btn.on_click(test_all_features)

def reset_dashboard(button=None):
    """Reset dashboard to initial state"""
    state.update({
        'df': None,
        'filtered_df': None,
        'history': [{"action": "Initial State", "timestamp": datetime.now().isoformat()}],
        'history_index': 0,
        'current_filters': {},
        'uploaded_file': None,
        'visualizations': {},
        'ml_model': None,
        'ml_results': None
    })

    # Clear outputs (safe: ignore those not widgets)
    for output in [upload_output]:
        try:
            with output:
                clear_output()
        except Exception:
            pass

    # Reset file info
    file_info.value = "<div style='color: var(--text-secondary);'>No file selected</div>"

    # Update status bar
    status_bar.value = create_status_bar().value

    # Show reset message
    with upload_output:
        clear_output()
        display(create_alert("Dashboard reset successfully! Ready for new data.", "info"))

reset_btn.on_click(reset_dashboard)

# Display upload section
#display(upload_section)

#print("✅ File upload system ready!")


In [34]:
#print("📊 Creating data profiling system...")

# Profile buttons
profile_button = Button(
    description='Generate Profile',
    button_style='primary',
    layout=Layout(width='200px', height='40px'),
    style={'font_weight': 'bold'}
)

clear_profile_button = Button(
    description='Clear Profile',
    button_style='warning',
    layout=Layout(width='150px', height='40px')
)

# Profile controls
profile_controls = HBox([profile_button, clear_profile_button])

# Profile section
profile_section = VBox([
    HTML("<h3 class='neon-text'>📊 Data Profiling & Statistics</h3>"),
    HTML("<p style='color: var(--text-secondary);'>Analyze dataset statistics, missing values, and data quality</p>"),
    profile_controls,
    profile_output
], layout=Layout(
    padding='20px',
    border='1px solid rgba(0, 243, 255, 0.2)',
    border_radius='12px',
    background='rgba(16, 18, 40, 0.3)'
))

def generate_profile(button=None):
    """Generate data profile"""
    with profile_output:
        clear_output()
        
        if state['df'] is None:
            display(create_alert("Please load data first", "warning"))
            return
        
        df = state['df']
        
        # Show loading
        display(create_loading_placeholder(150))
        
        try:
            # Calculate statistics
            numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
            categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
            
            # Create statistics display
            stats_html = f"""
            <div style="margin: 20px 0;">
                <h4 style="color: var(--neon-cyan);">📈 Dataset Overview</h4>
                <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin: 20px 0;">
                    <div class="stat-card">
                        <div style="font-size: 2rem; color: #00f3ff;">{len(df):,}</div>
                        <div style="color: var(--text-secondary);">Total Rows</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 2rem; color: #ff00ff;">{len(df.columns)}</div>
                        <div style="color: var(--text-secondary);">Total Columns</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 2rem; color: #00f3ff;">{len(numeric_cols)}</div>
                        <div style="color: var(--text-secondary);">Numeric Columns</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 2rem; color: #ff00ff;">{len(categorical_cols)}</div>
                        <div style="color: var(--text-secondary);">Categorical Columns</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 2rem; color: #00f3ff;">{df.isnull().sum().sum():,}</div>
                        <div style="color: var(--text-secondary);">Missing Values</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 2rem; color: #ff00ff;">{df.duplicated().sum():,}</div>
                        <div style="color: var(--text-secondary);">Duplicate Rows</div>
                    </div>
                </div>
            </div>
            """
            
            # Column details
            col_details = []
            for col in df.columns:
                col_type = str(df[col].dtype)
                missing = df[col].isnull().sum()
                missing_pct = (missing / len(df)) * 100
                unique = df[col].nunique()
                
                if col_type in ['int64', 'float64']:
                    col_stats = f"Mean: {df[col].mean():.2f}, Min: {df[col].min():.2f}, Max: {df[col].max():.2f}"
                else:
                    col_stats = f"Top: {df[col].mode().iloc[0] if not df[col].mode().empty else 'N/A'}"
                
                col_details.append(f"""
                <tr>
                    <td style="color: #00f3ff; font-weight: bold;">{col}</td>
                    <td>{col_type}</td>
                    <td>{missing} ({missing_pct:.1f}%)</td>
                    <td>{unique}</td>
                    <td>{col_stats[:50]}...</td>
                </tr>
                """)
            
            col_details_html = f"""
            <div style="margin: 20px 0;">
                <h4 style="color: var(--neon-cyan);">📋 Column Details</h4>
                <div style="overflow-x: auto; max-height: 400px;">
                    <table class="data-table" style="width: 100%;">
                        <thead>
                            <tr>
                                <th>Column</th>
                                <th>Type</th>
                                <th>Missing</th>
                                <th>Unique Values</th>
                                <th>Statistics</th>
                            </tr>
                        </thead>
                        <tbody>
                            {''.join(col_details)}
                        </tbody>
                    </table>
                </div>
            </div>
            """
            
            # Missing values heatmap
            missing_data = df.isnull().astype(int)
            missing_fig = go.Figure(data=go.Heatmap(
                z=missing_data.values.T,
                x=list(range(len(df))),
                y=df.columns.tolist(),
                colorscale=[[0, 'rgba(16, 18, 40, 0.8)'], [1, 'rgba(255, 0, 255, 0.8)']],
                showscale=True,
                hovertemplate='Column: %{y}<br>Row: %{x}<br>Missing: %{z}<extra></extra>'
            ))
            
            missing_fig.update_layout(
                title='Missing Values Heatmap',
                height=400,
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                font_color='white',
                xaxis_showgrid=False,
                yaxis_showgrid=False,
                margin=dict(l=50, r=50, t=50, b=50)
            )
            
            # Clear and display results
            clear_output()
            display(HTML(stats_html))
            display(HTML(col_details_html))
            display(missing_fig)
            
            # Add to history
            add_to_history("Generated data profile", {
                "numeric_columns": len(numeric_cols),
                "categorical_columns": len(categorical_cols),
                "missing_values": df.isnull().sum().sum()
            })
            
        except Exception as e:
            clear_output()
            display(create_alert(f"Error generating profile: {str(e)}", "error"))

def clear_profile(button):
    """Clear profile output"""
    with profile_output:
        clear_output()

# Connect buttons
profile_button.on_click(generate_profile)
clear_profile_button.on_click(clear_profile)

# Display profile section
#display(profile_section)

#print("✅ Data profiling system ready!")

In [35]:
#print("🔍 Creating filtering system...")

# Filter widgets storage
filter_widgets = {}

# Filter logic toggle
filter_logic = RadioButtons(
    options=['AND (All filters must match)', 'OR (Any filter can match)'],
    value='AND (All filters must match)',
    layout=Layout(width='400px')
)

# Filter controls
apply_filter_button = Button(
    description='Apply Filters',
    button_style='primary',
    layout=Layout(width='150px', height='40px'),
    style={'font_weight': 'bold'}
)

clear_filter_button = Button(
    description='Clear Filters',
    button_style='warning',
    layout=Layout(width='150px', height='40px')
)

reset_filter_button = Button(
    description='Reset All',
    button_style='danger',
    layout=Layout(width='150px', height='40px')
)

filter_controls = HBox([apply_filter_button, clear_filter_button, reset_filter_button])

# Filter results display
filter_stats = HTML("")

# Create filter creation function
def create_filter_widgets():
    """Create filter widgets based on current data"""
    if state['df'] is None:
        return HTML("<div style='color: var(--text-secondary); padding: 20px; text-align: center;'>No data loaded for filtering</div>")
    
    df = state['df']
    widgets_list = []
    
    # Search across all columns
    search_box = Text(
        placeholder='Search across all columns...',
        layout=Layout(width='100%', margin='10px 0')
    )
    widgets_list.append(VBox([
        HTML("<label style='color: var(--neon-cyan); font-weight: bold;'>🔍 Global Search</label>"),
        search_box
    ]))
    filter_widgets['_search'] = ('text', search_box)
    
    # Create widgets for each column
    for col in df.columns:
        col_type = str(df[col].dtype)
        
        if col_type in ['int64', 'float64']:
            # Numeric slider
            if not df[col].isnull().all():
                min_val = float(df[col].min())
                max_val = float(df[col].max())
                step = (max_val - min_val) / 100
                
                slider = FloatRangeSlider(
                    value=[min_val, max_val],
                    min=min_val,
                    max=max_val,
                    step=step,
                    description=col,
                    layout=Layout(width='95%'),
                    style={'description_width': '150px'}
                )
                widgets_list.append(slider)
                filter_widgets[col] = ('range', slider)
        
        elif df[col].nunique() < 20:  # Categorical with limited unique values
            unique_vals = sorted([str(v) for v in df[col].dropna().unique()])
            
            select = SelectMultiple(
                options=unique_vals,
                description=col,
                layout=Layout(width='95%'),
                style={'description_width': '150px'}
            )
            widgets_list.append(select)
            filter_widgets[col] = ('select', select)
        
        else:
            # Text filter for high-cardinality columns
            text_filter = Text(
                placeholder=f'Filter {col}...',
                description=col,
                layout=Layout(width='95%'),
                style={'description_width': '150px'}
            )
            widgets_list.append(text_filter)
            filter_widgets[col] = ('text', text_filter)
    
    return VBox(widgets_list, layout=Layout(
        max_height='400px',
        overflow_y='auto',
        padding='10px',
        border='1px solid rgba(0, 243, 255, 0.2)',
        border_radius='8px'
    ))

# Initial filter widgets
filter_widgets_container = create_filter_widgets()

# Filter section
filter_section = VBox([
    HTML("<h3 class='neon-text'>🔍 Dynamic Filter Engine</h3>"),
    HTML("<p style='color: var(--text-secondary);'>Apply filters with AND/OR logic, numeric sliders, and category selectors</p>"),
    HTML("<label style='color: var(--neon-cyan); font-weight: bold;'>Filter Logic:</label>"),
    filter_logic,
    HBox([
        VBox([
            HTML("<h4 style='color: var(--neon-cyan); margin-top: 0;'>Filter Controls</h4>"),
            filter_controls,
            filter_stats
        ], layout=Layout(width='30%', padding='10px')),
        VBox([
            HTML("<h4 style='color: var(--neon-cyan); margin-top: 0;'>Available Filters</h4>"),
            filter_widgets_container
        ], layout=Layout(width='70%', padding='10px'))
    ]),
    filter_output
], layout=Layout(
    padding='20px',
    border='1px solid rgba(0, 243, 255, 0.2)',
    border_radius='12px',
    background='rgba(16, 18, 40, 0.3)'
))

def apply_filters(button):
    """Apply all active filters"""
    with filter_output:
        clear_output()
        
        if state['df'] is None:
            display(create_alert("No data available for filtering", "warning"))
            return
        
        df = state['df']
        filtered_df = df.copy()
        
        # Get filter logic
        use_and = 'AND' in filter_logic.value
        
        # Apply each filter
        filters_applied = []
        
        # Global search
        if '_search' in filter_widgets:
            filter_type, widget = filter_widgets['_search']
            if widget.value:
                search_term = widget.value.lower()
                mask = pd.Series(False, index=filtered_df.index)
                for col in filtered_df.columns:
                    try:
                        mask = mask | filtered_df[col].astype(str).str.lower().str.contains(search_term, na=False)
                    except:
                        continue
                filtered_df = filtered_df[mask]
                filters_applied.append(f"Global search: '{widget.value}'")
        
        # Column-specific filters
        masks = []
        for col, (filter_type, widget) in filter_widgets.items():
            if col == '_search':
                continue
            
            try:
                if filter_type == 'range' and hasattr(widget, 'value'):
                    min_val, max_val = widget.value
                    mask = (filtered_df[col] >= min_val) & (filtered_df[col] <= max_val)
                    if not mask.all():  # If filter is not showing all data
                        masks.append(mask)
                        filters_applied.append(f"{col}: {min_val:.2f} to {max_val:.2f}")
                
                elif filter_type == 'select' and widget.value:
                    mask = filtered_df[col].isin(widget.value)
                    masks.append(mask)
                    filters_applied.append(f"{col}: {len(widget.value)} selected")
                
                elif filter_type == 'text' and widget.value:
                    mask = filtered_df[col].astype(str).str.contains(widget.value, case=False, na=False)
                    masks.append(mask)
                    filters_applied.append(f"{col} contains: '{widget.value}'")
            except Exception as e:
                print(f"Error applying filter for {col}: {e}")
                continue
        
        # Combine masks based on logic
        if masks:
            if use_and:  # AND logic
                combined_mask = pd.Series(True, index=filtered_df.index)
                for mask in masks:
                    combined_mask = combined_mask & mask
                filtered_df = filtered_df[combined_mask]
            else:  # OR logic
                combined_mask = pd.Series(False, index=filtered_df.index)
                for mask in masks:
                    combined_mask = combined_mask | mask
                filtered_df = filtered_df[combined_mask]
        
        # Update filtered dataset
        state['filtered_df'] = filtered_df
        
        # Update stats display
        filtered_rows = len(filtered_df)
        total_rows = len(df)
        filtered_out = total_rows - filtered_rows
        
        filter_stats.value = f"""
        <div style="
            background: rgba(0, 243, 255, 0.1);
            padding: 15px;
            border-radius: 8px;
            border-left: 4px solid #00f3ff;
            margin: 10px 0;
        ">
            <h4 style="color: #00f3ff; margin: 0 0 10px 0;">📊 Filter Results</h4>
            <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px;">
                <div class="stat-card">
                    <div style="font-size: 1.5rem; color: #00f3ff;">{filtered_rows:,}</div>
                    <div style="color: var(--text-secondary);">Rows After Filter</div>
                </div>
                <div class="stat-card">
                    <div style="font-size: 1.5rem; color: #ff00ff;">{filtered_out:,}</div>
                    <div style="color: var(--text-secondary);">Rows Filtered Out</div>
                </div>
                <div class="stat-card">
                    <div style="font-size: 1.5rem; color: #00f3ff;">{(filtered_rows/total_rows*100):.1f}%</div>
                    <div style="color: var(--text-secondary);">Data Remaining</div>
                </div>
            </div>
        </div>
        """
        
        # Show filtered data preview
        if filters_applied:
            filters_html = "<div style='margin: 10px 0;'><strong>Active Filters:</strong><br>"
            for f in filters_applied:
                filters_html += f"<span style='background: rgba(0, 243, 255, 0.1); padding: 5px 10px; margin: 2px; border-radius: 4px; display: inline-block;'>{f}</span> "
            filters_html += "</div>"
            display(HTML(filters_html))
        
        # Show preview
        if len(filtered_df) > 0:
            preview_html = filtered_df.head(10).to_html(classes='data-table')
            display(HTML(f"""
            <div style="margin: 20px 0;">
                <h4 style="color: var(--neon-cyan);">Filtered Data Preview:</h4>
                <div style="overflow-x: auto; max-height: 400px;">
                    {preview_html}
                </div>
            </div>
            """))
        else:
            display(create_alert("No data matches all filters", "warning"))
        
        # Add to history
        add_to_history("Applied filters", {
            "filters_applied": len(filters_applied),
            "rows_remaining": filtered_rows,
            "logic": "AND" if use_and else "OR"
        })

def clear_filters(button):
    """Clear all filters"""
    for col, (filter_type, widget) in filter_widgets.items():
        try:
            if filter_type == 'range':
                widget.value = [widget.min, widget.max]
            elif filter_type in ['select', 'text']:
                widget.value = None
        except:
            continue
    
    # Reset filtered dataset
    if state['df'] is not None:
        state['filtered_df'] = state['df'].copy()
    
    with filter_output:
        clear_output()
        display(create_alert("All filters cleared", "info"))
    
    filter_stats.value = ""

def reset_filters(button):
    """Reset filter widgets and clear output"""
    global filter_widgets_container
    filter_widgets.clear()
    filter_widgets_container = create_filter_widgets()
    
    # Update the display
    filter_section.children = list(filter_section.children[:-1]) + [filter_widgets_container, filter_output]
    
    with filter_output:
        clear_output()
        display(create_alert("Filter widgets reset", "info"))
    
    filter_stats.value = ""

# Connect buttons
apply_filter_button.on_click(apply_filters)
clear_filter_button.on_click(clear_filters)
reset_filter_button.on_click(reset_filters)

# Display filter section
#display(filter_section)

#print("✅ Filtering system ready!")

In [36]:
#print("📈 Creating visualization system...")

# Visualization type selector
viz_type = Dropdown(
    options=['Line Chart', 'Bar Chart', 'Scatter Plot', 'Histogram', 'Box Plot', 'Pie Chart', 'Heatmap', 'Violin Plot'],
    value='Line Chart',
    description='Chart Type:',
    layout=Layout(width='300px'),
    style={'description_width': '100px'}
)

# Column selectors
x_column = Dropdown(
    description='X Column:',
    layout=Layout(width='250px'),
    style={'description_width': '80px'}
)

y_column = Dropdown(
    description='Y Column:',
    layout=Layout(width='250px'),
    style={'description_width': '80px'}
)

color_column = Dropdown(
    description='Color By:',
    layout=Layout(width='250px'),
    style={'description_width': '80px'}
)

# Visualization controls
generate_viz_button = Button(
    description='Generate Chart',
    button_style='primary',
    layout=Layout(width='200px', height='40px'),
    style={'font_weight': 'bold'}
)

clear_viz_button = Button(
    description='Clear Chart',
    button_style='warning',
    layout=Layout(width='150px', height='40px')
)

download_viz_button = Button(
    description='Download PNG',
    button_style='success',
    layout=Layout(width='150px', height='40px')
)

viz_controls = HBox([generate_viz_button, clear_viz_button, download_viz_button])

# Update column selectors based on data
def update_column_selectors():
    """Update column dropdowns based on current data"""
    if state['filtered_df'] is not None:
        df = state['filtered_df']
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
        all_cols = df.columns.tolist()
        
        x_column.options = all_cols
        y_column.options = numeric_cols if numeric_cols else all_cols
        color_column.options = ['None'] + categorical_cols
        
        if all_cols:
            x_column.value = all_cols[0]
            if numeric_cols:
                y_column.value = numeric_cols[0] if len(numeric_cols) > 0 else all_cols[0]
            else:
                y_column.value = all_cols[0] if len(all_cols) > 1 else all_cols[0]
            color_column.value = 'None'

# Initialize column selectors
update_column_selectors()

# Visualization section
viz_section = VBox([
    HTML("<h3 class='neon-text'>📈 Interactive Visualizations</h3>"),
    HTML("<p style='color: var(--text-secondary);'>Create interactive charts with futuristic styling</p>"),
    
    # Chart controls
    HBox([
        VBox([
            HTML("<label style='color: var(--neon-cyan); font-weight: bold;'>Chart Configuration</label>"),
            viz_type,
            x_column,
            y_column,
            color_column
        ], layout=Layout(padding='10px')),
        
        VBox([
            HTML("<label style='color: var(--neon-cyan); font-weight: bold;'>Actions</label>"),
            viz_controls
        ], layout=Layout(padding='10px'))
    ], layout=Layout(
        border='1px solid rgba(0, 243, 255, 0.2)',
        border_radius='8px',
        padding='10px',
        margin='10px 0'
    )),
    
    viz_output
], layout=Layout(
    padding='20px',
    border='1px solid rgba(0, 243, 255, 0.2)',
    border_radius='12px',
    background='rgba(16, 18, 40, 0.3)'
))

def create_visualization(button):
    """Create visualization based on selected options"""
    with viz_output:
        clear_output()
        
        if state['filtered_df'] is None:
            display(create_alert("Please load and filter data first", "warning"))
            return
        
        df = state['filtered_df']
        
        # Show loading
        display(create_loading_placeholder(200))
        
        try:
            # Get selected columns
            x_col = x_column.value
            y_col = y_column.value
            color_col = color_column.value if color_column.value != 'None' else None
            
            # Create figure based on type
            fig = None
            chart_type = viz_type.value
            
            if chart_type == 'Line Chart':
                fig = px.line(
                    df, 
                    x=x_col, 
                    y=y_col,
                    color=color_col,
                    title=f'Line Chart: {y_col} vs {x_col}',
                    template='plotly_dark'
                )
                
            elif chart_type == 'Bar Chart':
                # Aggregate for bar chart
                if color_col:
                    agg_df = df.groupby([x_col, color_col])[y_col].mean().reset_index()
                    fig = px.bar(
                        agg_df,
                        x=x_col,
                        y=y_col,
                        color=color_col,
                        title=f'Bar Chart: {y_col} by {x_col}',
                        barmode='group',
                        template='plotly_dark'
                    )
                else:
                    agg_df = df.groupby(x_col)[y_col].mean().reset_index()
                    fig = px.bar(
                        agg_df,
                        x=x_col,
                        y=y_col,
                        title=f'Bar Chart: Average {y_col} by {x_col}',
                        template='plotly_dark'
                    )
                
            elif chart_type == 'Scatter Plot':
                fig = px.scatter(
                    df,
                    x=x_col,
                    y=y_col,
                    color=color_col,
                    title=f'Scatter Plot: {y_col} vs {x_col}',
                    template='plotly_dark',
                    hover_data=df.columns.tolist()[:5]
                )
                
            elif chart_type == 'Histogram':
                fig = px.histogram(
                    df,
                    x=y_col,
                    color=color_col,
                    title=f'Histogram: {y_col} Distribution',
                    template='plotly_dark',
                    nbins=30
                )
                
            elif chart_type == 'Box Plot':
                fig = px.box(
                    df,
                    y=y_col,
                    x=color_col if color_col else None,
                    title=f'Box Plot: {y_col} Distribution',
                    template='plotly_dark'
                )
                
            elif chart_type == 'Pie Chart':
                # Aggregate for pie chart
                agg_df = df.groupby(x_col)[y_col].sum().reset_index()
                fig = px.pie(
                    agg_df,
                    values=y_col,
                    names=x_col,
                    title=f'Pie Chart: {y_col} by {x_col}',
                    template='plotly_dark',
                    hole=0.3
                )
                
            elif chart_type == 'Heatmap':
                # Select numeric columns for correlation heatmap
                numeric_df = df.select_dtypes(include=[np.number])
                if len(numeric_df.columns) < 2:
                    display(create_alert("Need at least 2 numeric columns for heatmap", "warning"))
                    return
                
                corr_matrix = numeric_df.corr()
                fig = go.Figure(data=go.Heatmap(
                    z=corr_matrix.values,
                    x=corr_matrix.columns,
                    y=corr_matrix.index,
                    colorscale='Portland',
                    zmin=-1,
                    zmax=1,
                    colorbar=dict(title='Correlation')
                ))
                fig.update_layout(
                    title='Correlation Heatmap',
                    template='plotly_dark'
                )
                
            elif chart_type == 'Violin Plot':
                fig = px.violin(
                    df,
                    y=y_col,
                    x=color_col if color_col else None,
                    title=f'Violin Plot: {y_col} Distribution',
                    template='plotly_dark'
                )
            
            # Apply futuristic styling
            if fig:
                fig.update_layout(
                    template='plotly_dark',
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    font_color='white',
                    title_font_color='#00f3ff',
                    title_font_size=20,
                    hoverlabel=dict(
                        bgcolor='rgba(10, 10, 26, 0.9)',
                        font_color='white',
                        font_size=12
                    )
                )
                
                # Update traces for neon colors
                for i, trace in enumerate(fig.data):
                    if hasattr(trace, 'marker'):
                        if trace.marker is not None:
                            if hasattr(trace.marker, 'color'):
                                # Cycle through neon colors
                                colors = ['#00f3ff', '#ff00ff', '#9d00ff', '#00ff9d', '#ff9d00']
                                trace.marker.color = colors[i % len(colors)]
                    
                    if hasattr(trace, 'line'):
                        if trace.line is not None:
                            if hasattr(trace.line, 'color'):
                                trace.line.color = '#00f3ff'
                
                # Clear and display
                clear_output()
                display(fig)
                
                # Store visualization
                state['visualizations'][f"{chart_type}_{datetime.now().strftime('%H%M%S')}"] = fig
                
                # Add to history
                add_to_history(f"Created {chart_type}", {
                    "x_column": x_col,
                    "y_column": y_col,
                    "color_column": color_col
                })
                
            else:
                clear_output()
                display(create_alert(f"Could not create {chart_type} with selected columns", "error"))
                
        except Exception as e:
            clear_output()
            display(create_alert(f"Error creating visualization: {str(e)}", "error"))

def clear_visualization(button):
    """Clear visualization output"""
    with viz_output:
        clear_output()

def download_visualization(button):
    """Download current visualization as PNG"""
    with viz_output:
        if state['visualizations']:
            # Get the last visualization
            last_viz_key = list(state['visualizations'].keys())[-1]
            fig = state['visualizations'][last_viz_key]
            
            # Create filename
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"neonflux_chart_{timestamp}.png"
            
            # Save image
            fig.write_image(filename)
            
            # Create download link
            with open(filename, "rb") as f:
                b64 = base64.b64encode(f.read()).decode()
            
            download_html = f"""
            <div style="
                background: rgba(0, 255, 0, 0.1);
                padding: 15px;
                border-radius: 8px;
                border-left: 4px solid #00ff00;
                margin: 10px 0;
            ">
                <h4 style="color: #00ff00; margin: 0 0 10px 0;">✅ Chart Saved!</h4>
                <p style="margin: 5px 0;">Download: <a href="data:image/png;base64,{b64}" download="{filename}" style="color: #00f3ff; text-decoration: none; font-weight: bold;">{filename}</a></p>
                <p style="margin: 5px 0; color: var(--text-secondary);">Right-click the link and select "Save link as..."</p>
            </div>
            """
            
            display(HTML(download_html))
        else:
            display(create_alert("No visualization to download", "warning"))

# Connect buttons
generate_viz_button.on_click(create_visualization)
clear_viz_button.on_click(clear_visualization)
download_viz_button.on_click(download_visualization)

# Update selectors when data changes
def update_viz_selectors():
    update_column_selectors()

# Display visualization section
#display(viz_section)

#print("✅ Visualization system ready!")

In [37]:
#print("💡 Creating auto-insights generator...")

# Insights controls
generate_insights_button = Button(
    description='Generate Insights',
    button_style='primary',
    layout=Layout(width='200px', height='40px'),
    style={'font_weight': 'bold'}
)

clear_insights_button = Button(
    description='Clear Insights',
    button_style='warning',
    layout=Layout(width='150px', height='40px')
)

insights_controls = HBox([generate_insights_button, clear_insights_button])

# Insights section
insights_section = VBox([
    HTML("<h3 class='neon-text'>💡 Auto-Insights Generator</h3>"),
    HTML("<p style='color: var(--text-secondary);'>AI-powered insights on missing data, outliers, correlations, and data quality</p>"),
    insights_controls,
    insights_output
], layout=Layout(
    padding='20px',
    border='1px solid rgba(0, 243, 255, 0.2)',
    border_radius='12px',
    background='rgba(16, 18, 40, 0.3)'
))

def generate_insights(button=None):
    """Generate automated insights from data"""
    with insights_output:
        clear_output()
        
        if state['filtered_df'] is None:
            display(create_alert("Please load data first", "warning"))
            return
        
        df = state['filtered_df']
        
        # Show loading
        display(create_loading_placeholder(150))
        
        try:
            insights = []
            warnings = []
            recommendations = []
            
            # 1. Missing data analysis
            missing_cols = df.columns[df.isnull().any()].tolist()
            if missing_cols:
                missing_html = "<div class='warning-card'><h4>⚠️ Missing Data Detected</h4><ul>"
                for col in missing_cols[:5]:  # Limit to first 5
                    missing_pct = df[col].isnull().mean() * 100
                    if missing_pct > 50:
                        warnings.append(f"Column '{col}' has {missing_pct:.1f}% missing values - consider removing")
                        missing_html += f"<li><strong>{col}:</strong> {missing_pct:.1f}% missing (Critical)</li>"
                    elif missing_pct > 10:
                        insights.append(f"Column '{col}' has {missing_pct:.1f}% missing values")
                        recommendations.append(f"Consider imputation for '{col}'")
                        missing_html += f"<li><strong>{col}:</strong> {missing_pct:.1f}% missing</li>"
                    else:
                        missing_html += f"<li><strong>{col}:</strong> {missing_pct:.1f}% missing (Minor)</li>"
                missing_html += "</ul></div>"
            else:
                missing_html = "<div class='stat-card'><h4>✅ No Missing Data</h4><p>All columns are complete.</p></div>"
            
            # 2. Outlier detection
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            if len(numeric_cols) > 0:
                outliers_html = "<div class='insight-card'><h4>📊 Outlier Analysis</h4><ul>"
                for col in numeric_cols[:3]:  # Check first 3 numeric columns
                    if df[col].notna().sum() > 10:
                        Q1 = df[col].quantile(0.25)
                        Q3 = df[col].quantile(0.75)
                        IQR = Q3 - Q1
                        lower_bound = Q1 - 1.5 * IQR
                        upper_bound = Q3 + 1.5 * IQR
                        outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
                        if outliers > 0:
                            pct_outliers = outliers / len(df) * 100
                            insights.append(f"Column '{col}' has {outliers} outliers ({pct_outliers:.1f}%)")
                            outliers_html += f"<li><strong>{col}:</strong> {outliers} outliers ({pct_outliers:.1f}%)</li>"
                            
                            if pct_outliers > 5:
                                warnings.append(f"High outlier percentage in '{col}' ({pct_outliers:.1f}%)")
                                recommendations.append(f"Investigate outliers in '{col}'")
                        else:
                            outliers_html += f"<li><strong>{col}:</strong> No outliers detected</li>"
                outliers_html += "</ul></div>"
            else:
                outliers_html = "<div class='stat-card'><h4>📊 No Numeric Columns</h4><p>Outlier analysis requires numeric data.</p></div>"
            
            # 3. Correlation insights
            if len(numeric_cols) >= 2:
                corr_matrix = df[numeric_cols].corr()
                high_corr_pairs = []
                for i in range(len(numeric_cols)):
                    for j in range(i+1, len(numeric_cols)):
                        corr_val = abs(corr_matrix.iloc[i, j])
                        if corr_val > 0.8:
                            col1 = numeric_cols[i]
                            col2 = numeric_cols[j]
                            high_corr_pairs.append(f"{col1} - {col2}: {corr_val:.2f}")
                
                if high_corr_pairs:
                    corr_html = "<div class='warning-card'><h4>🔗 High Correlations</h4><ul>"
                    for pair in high_corr_pairs[:3]:  # Show top 3
                        corr_html += f"<li>{pair}</li>"
                        recommendations.append(f"Check multicollinearity: {pair.split(':')[0]}")
                    corr_html += "</ul><p>These columns may be measuring similar things.</p></div>"
                    insights.append(f"Found {len(high_corr_pairs)} highly correlated column pairs")
                else:
                    corr_html = "<div class='stat-card'><h4>🔗 Correlation Analysis</h4><p>No concerning correlations detected.</p></div>"
            else:
                corr_html = "<div class='stat-card'><h4>🔗 Correlation Analysis</h4><p>Need at least 2 numeric columns for correlation analysis.</p></div>"
            
            # 4. Data quality
            duplicate_rows = df.duplicated().sum()
            if duplicate_rows > 0:
                dup_html = f"<div class='warning-card'><h4>⚠️ Duplicate Data</h4><p>Found {duplicate_rows} duplicate rows ({duplicate_rows/len(df)*100:.1f}% of data).</p><p>Recommendation: Remove duplicates for cleaner analysis.</p></div>"
                warnings.append(f"Found {duplicate_rows} duplicate rows")
                recommendations.append("Remove duplicate rows")
            else:
                dup_html = "<div class='stat-card'><h4>✅ No Duplicate Rows</h4><p>All rows are unique.</p></div>"
            
            # 5. Data distribution insights
            if len(numeric_cols) > 0:
                dist_html = "<div class='insight-card'><h4>📈 Distribution Analysis</h4><ul>"
                for col in numeric_cols[:2]:  # Check first 2 columns
                    skew_val = df[col].skew()
                    if abs(skew_val) > 1:
                        skew_type = "right" if skew_val > 0 else "left"
                        insights.append(f"Column '{col}' is {skew_type}-skewed ({skew_val:.2f})")
                        dist_html += f"<li><strong>{col}:</strong> {skew_type}-skewed (skewness: {skew_val:.2f})</li>"
                        
                        if abs(skew_val) > 2:
                            recommendations.append(f"Consider transformation for '{col}' (high skew: {skew_val:.2f})")
                    else:
                        dist_html += f"<li><strong>{col}:</strong> Normally distributed</li>"
                dist_html += "</ul></div>"
            else:
                dist_html = ""
            
            # Clear and display insights
            clear_output()
            
            # Summary stats
            stats_html = f"""
            <div style="margin: 20px 0;">
                <h4 style="color: var(--neon-cyan);">✨ Insights Summary</h4>
                <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin: 20px 0;">
                    <div class="stat-card">
                        <div style="font-size: 1.5rem; color: #00f3ff;">{len(insights)}</div>
                        <div style="color: var(--text-secondary);">Key Insights</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 1.5rem; color: #ff00ff;">{len(warnings)}</div>
                        <div style="color: var(--text-secondary);">Warnings</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 1.5rem; color: #00f3ff;">{len(recommendations)}</div>
                        <div style="color: var(--text-secondary);">Recommendations</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 1.5rem; color: #ff00ff;">{len(missing_cols)}</div>
                        <div style="color: var(--text-secondary);">Columns with Missing Data</div>
                    </div>
                </div>
            </div>
            """
            
            display(HTML(stats_html))
            display(HTML(missing_html))
            display(HTML(outliers_html))
            display(HTML(corr_html))
            display(HTML(dup_html))
            if dist_html:
                display(HTML(dist_html))
            
            # Show recommendations if any
            if recommendations:
                rec_html = "<div class='insight-card'><h4>💡 Recommendations</h4><ul>"
                for i, rec in enumerate(recommendations[:5]):  # Show top 5
                    rec_html += f"<li>{rec}</li>"
                rec_html += "</ul></div>"
                display(HTML(rec_html))
            
            # Add to history
            add_to_history("Generated insights", {
                "insights_count": len(insights),
                "warnings_count": len(warnings),
                "recommendations_count": len(recommendations)
            })
            
        except Exception as e:
            clear_output()
            display(create_alert(f"Error generating insights: {str(e)}", "error"))

def clear_insights(button):
    """Clear insights output"""
    with insights_output:
        clear_output()

# Connect buttons
generate_insights_button.on_click(generate_insights)
clear_insights_button.on_click(clear_insights)

# Display insights section
#display(insights_section)

#print("✅ Auto-insights generator ready!")

In [38]:
#print("🤖 Creating ML pipeline...")

# ML controls
target_column = Dropdown(
    description='Target:',
    layout=Layout(width='250px'),
    style={'description_width': '80px'}
)

ml_algorithm = Dropdown(
    options=['Logistic Regression', 'Random Forest', 'Gradient Boosting'],
    value='Logistic Regression',
    description='Algorithm:',
    layout=Layout(width='250px'),
    style={'description_width': '100px'}
)

run_ml_button = Button(
    description='Run ML Pipeline',
    button_style='primary',
    layout=Layout(width='200px', height='40px'),
    style={'font_weight': 'bold'}
)

clear_ml_button = Button(
    description='Clear Results',
    button_style='warning',
    layout=Layout(width='150px', height='40px')
)

ml_controls = HBox([run_ml_button, clear_ml_button])

# Update target column selector
def update_ml_selectors():
    """Update ML selectors based on data"""
    if state['filtered_df'] is not None:
        df = state['filtered_df']
        
        # For binary classification, find suitable target
        suitable_targets = []
        for col in df.columns:
            if df[col].nunique() <= 10:  # Columns with <= 10 unique values
                suitable_targets.append(col)
        
        target_column.options = suitable_targets if suitable_targets else df.columns.tolist()
        if suitable_targets:
            target_column.value = suitable_targets[0]

# Initialize selectors
update_ml_selectors()

# ML section
ml_section = VBox([
    HTML("<h3 class='neon-text'>🤖 Machine Learning Pipeline</h3>"),
    HTML("<p style='color: var(--text-secondary);'>Binary classification with multiple algorithms and performance metrics</p>"),
    
    # ML configuration
    HBox([
        VBox([
            HTML("<label style='color: var(--neon-cyan); font-weight: bold;'>Model Configuration</label>"),
            target_column,
            ml_algorithm,
            HTML("<p style='color: var(--text-secondary); font-size: 0.9rem;'>Target should be binary or low-cardinality for classification</p>")
        ], layout=Layout(padding='10px')),
        
        VBox([
            HTML("<label style='color: var(--neon-cyan); font-weight: bold;'>Actions</label>"),
            ml_controls
        ], layout=Layout(padding='10px'))
    ], layout=Layout(
        border='1px solid rgba(0, 243, 255, 0.2)',
        border_radius='8px',
        padding='10px',
        margin='10px 0'
    )),
    
    ml_output
], layout=Layout(
    padding='20px',
    border='1px solid rgba(0, 243, 255, 0.2)',
    border_radius='12px',
    background='rgba(16, 18, 40, 0.3)'
))

def run_ml_pipeline(button):
    """Run ML pipeline"""
    with ml_output:
        clear_output()
        
        if state['filtered_df'] is None:
            display(create_alert("Please load data first", "warning"))
            return
        
        df = state['filtered_df'].copy()
        
        # Show loading
        display(create_loading_placeholder(200))
        
        try:
            # Get target column
            target = target_column.value
            if target not in df.columns:
                display(create_alert(f"Target column '{target}' not found in data", "error"))
                return
            
            # Prepare features and target
            X = df.drop(columns=[target])
            y = df[target]
            
            # Handle missing values
            imputer = SimpleImputer(strategy='median')
            numeric_cols = X.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                X[numeric_cols] = imputer.fit_transform(X[numeric_cols])
            
            # Encode categorical features
            categorical_cols = X.select_dtypes(exclude=[np.number]).columns
            for col in categorical_cols:
                X[col] = LabelEncoder().fit_transform(X[col].astype(str))
            
            # Encode target if categorical
            if y.dtype == 'object':
                y = LabelEncoder().fit_transform(y.astype(str))
            
            # Check if target is binary or needs to be binarized
            unique_classes = np.unique(y)
            if len(unique_classes) > 2:
                # Convert to binary by taking median split
                median_val = np.median(y)
                y_binary = (y > median_val).astype(int)
                y = y_binary
                display(create_alert(f"Target has {len(unique_classes)} classes. Converted to binary using median split.", "info"))
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.3, random_state=42, stratify=y
            )
            
            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            # Select and train model
            algorithm = ml_algorithm.value
            if algorithm == 'Logistic Regression':
                model = LogisticRegression(random_state=42, max_iter=1000)
            elif algorithm == 'Random Forest':
                model = RandomForestClassifier(random_state=42, n_estimators=100)
            else:  # Gradient Boosting
                from sklearn.ensemble import GradientBoostingClassifier
                model = GradientBoostingClassifier(random_state=42, n_estimators=100)
            
            model.fit(X_train_scaled, y_train)
            
            # Make predictions
            y_pred = model.predict(X_test_scaled)
            y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
            recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
            f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
            
            # ROC Curve
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            roc_auc = auc(fpr, tpr)
            
            # Store model and results
            state['ml_model'] = model
            state['ml_results'] = {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'roc_auc': roc_auc
            }
            
            # Clear and display results
            clear_output()
            
            # Metrics display
            metrics_html = f"""
            <div style="margin: 20px 0;">
                <h4 style="color: var(--neon-cyan);">📊 Model Performance</h4>
                <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin: 20px 0;">
                    <div class="stat-card">
                        <div style="font-size: 1.5rem; color: #00f3ff;">{accuracy:.3f}</div>
                        <div style="color: var(--text-secondary);">Accuracy</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 1.5rem; color: #ff00ff;">{precision:.3f}</div>
                        <div style="color: var(--text-secondary);">Precision</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 1.5rem; color: #00f3ff;">{recall:.3f}</div>
                        <div style="color: var(--text-secondary);">Recall</div>
                    </div>
                    <div class="stat-card">
                        <div style="font-size: 1.5rem; color: #ff00ff;">{f1:.3f}</div>
                        <div style="color: var(--text-secondary);">F1 Score</div>
                    </div>
                </div>
            </div>
            
            <div class="stat-card">
                <h4>🧠 Model Information</h4>
                <p><strong>Algorithm:</strong> {algorithm}</p>
                <p><strong>Target:</strong> {target}</p>
                <p><strong>Features:</strong> {len(X.columns)}</p>
                <p><strong>Training samples:</strong> {len(X_train):,}</p>
                <p><strong>Test samples:</strong> {len(X_test):,}</p>
                <p><strong>ROC AUC:</strong> {roc_auc:.3f}</p>
            </div>
            """
            
            display(HTML(metrics_html))
            
            # ROC Curve
            roc_fig = go.Figure()
            roc_fig.add_trace(go.Scatter(
                x=fpr, y=tpr,
                mode='lines',
                line=dict(color='#00f3ff', width=3),
                name=f'ROC curve (AUC = {roc_auc:.3f})'
            ))
            roc_fig.add_trace(go.Scatter(
                x=[0, 1], y=[0, 1],
                mode='lines',
                line=dict(color='white', width=2, dash='dash'),
                name='Random classifier'
            ))
            
            roc_fig.update_layout(
                title='ROC Curve',
                xaxis_title='False Positive Rate',
                yaxis_title='True Positive Rate',
                template='plotly_dark',
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                height=400,
                margin=dict(l=50, r=50, t=50, b=50)
            )
            
            display(roc_fig)
            
            # Feature importance (if available)
            if hasattr(model, 'feature_importances_'):
                importance_df = pd.DataFrame({
                    'feature': X.columns,
                    'importance': model.feature_importances_
                }).sort_values('importance', ascending=False).head(10)
                
                importance_fig = go.Figure(data=[
                    go.Bar(
                        x=importance_df['importance'],
                        y=importance_df['feature'],
                        orientation='h',
                        marker_color='#9d00ff'
                    )
                ])
                
                importance_fig.update_layout(
                    title='Top 10 Feature Importances',
                    template='plotly_dark',
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    height=400,
                    margin=dict(l=150, r=50, t=50, b=50)
                )
                
                display(importance_fig)
            elif hasattr(model, 'coef_'):
                # For logistic regression
                coef_df = pd.DataFrame({
                    'feature': X.columns,
                    'coefficient': model.coef_[0]
                }).sort_values('coefficient', key=abs, ascending=False).head(10)
                
                coef_fig = go.Figure(data=[
                    go.Bar(
                        x=coef_df['coefficient'],
                        y=coef_df['feature'],
                        orientation='h',
                        marker_color='#00f3ff'
                    )
                ])
                
                coef_fig.update_layout(
                    title='Top 10 Feature Coefficients',
                    template='plotly_dark',
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    height=400,
                    margin=dict(l=150, r=50, t=50, b=50)
                )
                
                display(coef_fig)
            
            # Add to history
            add_to_history(f"Ran ML pipeline ({algorithm})", {
                "target": target,
                "algorithm": algorithm,
                "accuracy": accuracy,
                "features": len(X.columns)
            })
            
        except Exception as e:
            clear_output()
            display(create_alert(f"Error running ML pipeline: {str(e)}", "error"))

def clear_ml_results(button):
    """Clear ML output"""
    with ml_output:
        clear_output()

# Connect buttons
run_ml_button.on_click(run_ml_pipeline)
clear_ml_button.on_click(clear_ml_results)

# Display ML section
#display(ml_section)

#print("✅ ML pipeline ready!")

In [39]:
#print("🔄 Creating history and export systems...")

# History controls
update_history_button = Button(
    description='Refresh History',
    button_style='primary',
    layout=Layout(width='150px', height='40px')
)

undo_button = Button(
    description='↩️ Undo',
    button_style='warning',
    layout=Layout(width='100px', height='40px')
)

redo_button = Button(
    description='↪️ Redo',
    button_style='warning',
    layout=Layout(width='100px', height='40px')
)

clear_history_button = Button(
    description='Clear History',
    button_style='danger',
    layout=Layout(width='150px', height='40px')
)

history_controls = HBox([update_history_button, undo_button, redo_button, clear_history_button])

# Export controls
export_format = Dropdown(
    options=['CSV', 'Excel', 'JSON'],
    value='CSV',
    description='Format:',
    layout=Layout(width='150px'),
    style={'description_width': '60px'}
)

export_data_button = Button(
    description='Export Data',
    button_style='success',
    layout=Layout(width='150px', height='40px')
)

export_viz_button = Button(
    description='Export Charts',
    button_style='info',
    layout=Layout(width='150px', height='40px')
)

export_controls = HBox([export_format, export_data_button, export_viz_button])

# History section
history_section = VBox([
    HTML("<h3 class='neon-text'>🔄 Action History</h3>"),
    HTML("<p style='color: var(--text-secondary);'>Track all actions with undo/redo capability</p>"),
    history_controls,
    history_output
], layout=Layout(
    padding='20px',
    border='1px solid rgba(0, 243, 255, 0.2)',
    border_radius='12px',
    background='rgba(16, 18, 40, 0.3)',
    margin='0 0 20px 0'
))

# Export section
export_section = VBox([
    HTML("<h3 class='neon-text'>📤 Export Functions</h3>"),
    HTML("<p style='color: var(--text-secondary);'>Export filtered data and visualizations in multiple formats</p>"),
    export_controls,
    export_output
], layout=Layout(
    padding='20px',
    border='1px solid rgba(0, 243, 255, 0.2)',
    border_radius='12px',
    background='rgba(16, 18, 40, 0.3)'
))

def update_history_display(button=None):
    """Update history display"""
    with history_output:
        clear_output()
        
        if not state['history']:
            display(HTML("<div style='color: var(--text-secondary); text-align: center; padding: 20px;'>No history recorded yet</div>"))
            return
        
        # Create history items
        history_html = "<div style='max-height: 400px; overflow-y: auto;'>"
        
        for i, record in enumerate(state['history']):
            is_current = (i == state['history_index'])
            bg_color = 'rgba(0, 243, 255, 0.1)' if is_current else 'rgba(16, 18, 40, 0.5)'
            border_color = '#00f3ff' if is_current else 'rgba(0, 243, 255, 0.2)'
            
            history_html += f"""
            <div style="
                background: {bg_color};
                border-left: 4px solid {border_color};
                padding: 10px 15px;
                margin: 5px 0;
                border-radius: 4px;
            ">
                <div style="display: flex; justify-content: space-between; align-items: center;">
                    <div>
                        <span style="color: #00f3ff; font-weight: bold;">{i+1}.</span>
                        <span style="color: white; margin-left: 10px;">{record['action']}</span>
                    </div>
                    <div style="color: var(--text-secondary); font-size: 0.9rem;">
                        {record['timestamp']}
                    </div>
                </div>
            </div>
            """
        
        history_html += "</div>"
        
        # Add stats
        stats_html = f"""
        <div style="
            background: rgba(157, 0, 255, 0.1);
            padding: 15px;
            border-radius: 8px;
            border-left: 4px solid #9d00ff;
            margin: 10px 0;
        ">
            <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px;">
                <div class="stat-card">
                    <div style="font-size: 1.2rem; color: #00f3ff;">{len(state['history'])}</div>
                    <div style="color: var(--text-secondary); font-size: 0.9rem;">Total Actions</div>
                </div>
                <div class="stat-card">
                    <div style="font-size: 1.2rem; color: #ff00ff;">{state['history_index'] + 1}</div>
                    <div style="color: var(--text-secondary); font-size: 0.9rem;">Current Position</div>
                </div>
                <div class="stat-card">
                    <div style="font-size: 1.2rem; color: #00f3ff;">{len(state['history']) - state['history_index'] - 1}</div>
                    <div style="color: var(--text-secondary); font-size: 0.9rem;">Redo Available</div>
                </div>
            </div>
        </div>
        """
        
        display(HTML(stats_html + history_html))

def perform_undo(button):
    """Perform undo action"""
    if undo_action():
        update_history_display()
        with history_output:
            display(create_alert("Undo performed", "info"))
    else:
        with history_output:
            display(create_alert("Nothing to undo", "warning"))

def perform_redo(button):
    """Perform redo action"""
    if redo_action():
        update_history_display()
        with history_output:
            display(create_alert("Redo performed", "info"))
    else:
        with history_output:
            display(create_alert("Nothing to redo", "warning"))

def clear_history(button):
    """Clear history (keep initial state)"""
    if len(state['history']) > 1:
        state['history'] = [state['history'][0]]
        state['history_index'] = 0
        update_history_display()
        with history_output:
            display(create_alert("History cleared", "info"))
    else:
        with history_output:
            display(create_alert("History is already empty", "warning"))

# Export functions
def export_data(button):
    """Export filtered data"""
    with export_output:
        clear_output()
        
        if state['filtered_df'] is None:
            display(create_alert("No data to export", "warning"))
            return
        
        df = state['filtered_df']
        format_type = export_format.value
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        try:
            if format_type == 'CSV':
                filename = f"neonflux_export_{timestamp}.csv"
                df.to_csv(filename, index=False)
                content = df.to_csv(index=False)
                mime_type = 'text/csv'
                
            elif format_type == 'Excel':
                filename = f"neonflux_export_{timestamp}.xlsx"
                df.to_excel(filename, index=False)
                
                # Read file for download
                with open(filename, 'rb') as f:
                    content = f.read()
                mime_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                
            elif format_type == 'JSON':
                filename = f"neonflux_export_{timestamp}.json"
                df.to_json(filename, orient='records', indent=2)
                
                with open(filename, 'r') as f:
                    content = f.read()
                mime_type = 'application/json'
            
            # Create download link
            if format_type != 'Excel':
                b64 = base64.b64encode(content.encode()).decode()
            else:
                b64 = base64.b64encode(content).decode()
            
            download_html = f"""
            <div style="
                background: rgba(0, 255, 0, 0.1);
                padding: 15px;
                border-radius: 8px;
                border-left: 4px solid #00ff00;
                margin: 10px 0;
            ">
                <h4 style="color: #00ff00; margin: 0 0 10px 0;">✅ Export Successful!</h4>
                <p style="margin: 5px 0;"><strong>Format:</strong> {format_type}</p>
                <p style="margin: 5px 0;"><strong>Rows:</strong> {len(df):,}</p>
                <p style="margin: 5px 0;"><strong>Columns:</strong> {len(df.columns)}</p>
                <p style="margin: 10px 0;">
                    Download: 
                    <a href="data:{mime_type};base64,{b64}" download="{filename}" 
                       style="color: #00f3ff; text-decoration: none; font-weight: bold; padding: 5px 10px; background: rgba(0, 243, 255, 0.1); border-radius: 4px;">
                       {filename}
                    </a>
                </p>
            </div>
            """
            
            display(HTML(download_html))
            
            # Add to history
            add_to_history(f"Exported data as {format_type}", {
                "filename": filename,
                "rows": len(df),
                "columns": len(df.columns)
            })
            
        except Exception as e:
            display(create_alert(f"Error exporting data: {str(e)}", "error"))

def export_visualizations(button):
    """Export all visualizations"""
    with export_output:
        clear_output()
        
        if not state['visualizations']:
            display(create_alert("No visualizations to export", "warning"))
            return
        
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            # Create HTML report
            html_content = """
            <!DOCTYPE html>
            <html>
            <head>
                <title>NeonFlux Visualization Export</title>
                <style>
                    body { font-family: Arial, sans-serif; margin: 20px; background: #0a0a1a; color: white; }
                    .chart { margin: 30px 0; border: 1px solid #00f3ff; padding: 20px; border-radius: 8px; background: rgba(16, 18, 40, 0.7); }
                    h1 { color: #00f3ff; }
                    .meta { color: #a0a0c0; margin: 10px 0; }
                </style>
            </head>
            <body>
                <h1>⚡ NeonFlux Visualization Export</h1>
                <div class="meta">Generated: """ + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + """</div>
                <div class="meta">Total Charts: """ + str(len(state['visualizations'])) + """</div>
            """
            
            # Save each chart as PNG and add to HTML
            for i, (name, fig) in enumerate(state['visualizations'].items()):
                png_filename = f"chart_{i+1}_{timestamp}.png"
                fig.write_image(png_filename)
                
                # Convert PNG to base64 for embedding
                with open(png_filename, "rb") as f:
                    png_b64 = base64.b64encode(f.read()).decode()
                
                html_content += f"""
                <div class="chart">
                    <h2>Chart {i+1}: {name}</h2>
                    <img src="data:image/png;base64,{png_b64}" style="max-width: 100%;" />
                </div>
                """
            
            html_content += "</body></html>"
            
            # Save HTML file
            html_filename = f"neonflux_visualizations_{timestamp}.html"
            with open(html_filename, 'w') as f:
                f.write(html_content)
            
            # Create download link for HTML
            html_b64 = base64.b64encode(html_content.encode()).decode()
            
            download_html = f"""
            <div style="
                background: rgba(0, 243, 255, 0.1);
                padding: 15px;
                border-radius: 8px;
                border-left: 4px solid #00f3ff;
                margin: 10px 0;
            ">
                <h4 style="color: #00f3ff; margin: 0 0 10px 0;">✅ Visualizations Exported!</h4>
                <p style="margin: 5px 0;"><strong>Total Charts:</strong> {len(state['visualizations'])}</p>
                <p style="margin: 5px 0;"><strong>HTML Report:</strong> 
                    <a href="data:text/html;base64,{html_b64}" download="{html_filename}" 
                       style="color: #00f3ff; text-decoration: none; font-weight: bold;">
                       {html_filename}
                    </a>
                </p>
                <p style="margin: 5px 0; color: var(--text-secondary);">Individual PNG files have also been saved.</p>
            </div>
            """
            
            display(HTML(download_html))
            
            # Add to history
            add_to_history("Exported visualizations", {
                "chart_count": len(state['visualizations']),
                "html_file": html_filename
            })
            
        except Exception as e:
            display(create_alert(f"Error exporting visualizations: {str(e)}", "error"))

# Connect buttons
update_history_button.on_click(update_history_display)
undo_button.on_click(perform_undo)
redo_button.on_click(perform_redo)
clear_history_button.on_click(clear_history)
export_data_button.on_click(export_data)
export_viz_button.on_click(export_visualizations)

# Initial history display
update_history_display()

# Display sections
#display(history_section)
#display(export_section)

#print("✅ History and export systems ready!")

In [40]:
#print("🚀 Assembling final dashboard with tabs...")

# Create a tab widget for all sections
tabs = Tab()

# Set tab contents
tabs.children = [
    upload_section,
    profile_section,
    filter_section,
    viz_section,
    insights_section,
    ml_section,
    history_section,
    export_section
]

# Set tab titles
tab_titles = [
    '📁 Data Upload',
    '📊 Data Profiling',
    '🔍 Filter Engine',
    '📈 Visualizations',
    '💡 Auto-Insights',
    '🤖 ML Pipeline',
    '🔄 History',
    '📤 Export'
]

for i, title in enumerate(tab_titles):
    tabs.set_title(i, title)

# Add custom tab styling
display(HTML("""
<style>
    /* Custom tab styling */
    .p-TabBar {
        background: rgba(16, 18, 40, 0.9) !important;
        border-bottom: 2px solid rgba(0, 243, 255, 0.3) !important;
        padding: 5px !important;
    }
    
    .p-TabBar-tab {
        color: #a0a0c0 !important;
        background: rgba(10, 10, 26, 0.7) !important;
        border: 1px solid rgba(0, 243, 255, 0.2) !important;
        border-bottom: none !important;
        margin-right: 5px !important;
        padding: 10px 20px !important;
        font-family: 'Exo 2', sans-serif !important;
        font-weight: 600 !important;
        border-radius: 8px 8px 0 0 !important;
        transition: all 0.3s ease !important;
    }
    
    .p-TabBar-tab.p-mod-current {
        color: #00f3ff !important;
        background: rgba(0, 243, 255, 0.1) !important;
        border-color: #00f3ff !important;
        box-shadow: 0 0 15px rgba(0, 243, 255, 0.3) !important;
        transform: translateY(-2px) !important;
    }
    
    .p-TabBar-tab:hover:not(.p-mod-current) {
        color: #ff00ff !important;
        background: rgba(255, 0, 255, 0.1) !important;
        border-color: rgba(255, 0, 255, 0.3) !important;
    }
    
    .p-TabPanel {
        background: rgba(16, 18, 40, 0.7) !important;
        border: 1px solid rgba(0, 243, 255, 0.2) !important;
        border-top: none !important;
        padding: 0 !important;
        border-radius: 0 0 8px 8px !important;
        overflow: hidden !important;
    }
</style>
"""))

# Create tab change handler to update selectors when switching tabs
def on_tab_change(change):
    """Handle tab changes"""
    if change['name'] == 'selected_index':
        # Update selectors when switching to certain tabs
        if change['new'] == 1:  # Profiling tab
            pass  # Auto-generate profile if data exists
        elif change['new'] == 2:  # Filter tab
            # Update filter widgets
            global filter_widgets_container
            filter_widgets_container = create_filter_widgets()
        elif change['new'] == 3:  # Visualization tab
            update_column_selectors()
        elif change['new'] == 5:  # ML tab
            update_ml_selectors()

# Observe tab changes
tabs.observe(on_tab_change, names='selected_index')

# Create footer
footer = HTML("""
<div style="
    margin-top: 30px;
    padding: 20px;
    background: rgba(10, 10, 26, 0.8);
    border-radius: 8px;
    border: 1px solid rgba(0, 243, 255, 0.2);
    text-align: center;
    color: var(--text-secondary);
">
    <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
        <div>
            <strong>⚡ NeonFlux Dashboard v2.0</strong> | Futuristic Data Insights Platform
        </div>
        <div>
            Built with Python • Plotly • ipywidgets
        </div>
        <div>
            © 2024 NeonFlux Analytics
        </div>
    </div>
    <div style="margin-top: 10px; font-size: 0.9rem; color: rgba(160, 160, 192, 0.7);">
        Ready for Voilà deployment | All features self-contained in notebook
    </div>
</div>
""")

# Clear previous output and display final dashboard
clear_output()

# Display everything in order
display(header)
display(status_bar)
display(quick_actions)

# Add instructions
instructions = HTML("""
<div style="
    background: rgba(157, 0, 255, 0.1);
    padding: 20px;
    border-radius: 8px;
    border-left: 4px solid #9d00ff;
    margin: 20px 0;
">
    <h3 style="color: #9d00ff; margin-top: 0;">🚀 Getting Started</h3>
    
    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 15px; margin: 15px 0;">
        <div style="background: rgba(0, 243, 255, 0.05); padding: 15px; border-radius: 6px;">
            <h4 style="color: #00f3ff; margin: 0 0 10px 0;">1. Load Data</h4>
            <p style="margin: 5px 0; color: var(--text-secondary);">• Click "Load Sample Data" for instant demo</p>
            <p style="margin: 5px 0; color: var(--text-secondary);">• Or upload your own CSV/Excel in Upload tab</p>
        </div>
        
        <div style="background: rgba(255, 0, 255, 0.05); padding: 15px; border-radius: 6px;">
            <h4 style="color: #ff00ff; margin: 0 0 10px 0;">2. Explore & Analyze</h4>
            <p style="margin: 5px 0; color: var(--text-secondary);">• Check Data Profiling for statistics</p>
            <p style="margin: 5px 0; color: var(--text-secondary);">• Use Filter Engine to focus on data</p>
        </div>
        
        <div style="background: rgba(0, 243, 255, 0.05); padding: 15px; border-radius: 6px;">
            <h4 style="color: #00f3ff; margin: 0 0 10px 0;">3. Visualize & Model</h4>
            <p style="margin: 5px 0; color: var(--text-secondary);">• Create charts in Visualizations</p>
            <p style="margin: 5px 0; color: var(--text-secondary);">• Run ML models in ML Pipeline</p>
        </div>
    </div>
    
    <div style="margin-top: 15px; padding-top: 15px; border-top: 1px solid rgba(0, 243, 255, 0.2);">
        <p style="margin: 0; color: white;">
            <strong>💡 Tip:</strong> Click on any tab above to switch between dashboard sections. 
            Each tab contains specialized tools for different analysis stages.
        </p>
    </div>
</div>
""")

display(instructions)
display(tabs)
display(footer)




HTML(value="<div style='padding:6px; font-size:0.9rem; color:var(--text-secondary)'>Status: rows=0 cols=0</div…

HTML(value='\n<div style="\n    background: rgba(157, 0, 255, 0.1);\n    padding: 20px;\n    border-radius: 8p…

HTML(value='\n<div style="\n    margin-top: 30px;\n    padding: 20px;\n    background: rgba(10, 10, 26, 0.8);\…